In [2]:
import pandas as pd

In [3]:
df_inp = pd.read_csv('nse_sensex.csv')

In [21]:
df_inp['DATE'] = pd.to_datetime(df_inp['DATE'])

In [22]:
df_inp.head()

,DATE,SYMBOL,SERIES,PREV CLOSE,OPEN,HIGH,LOW,LAST,CLOSE,VWAP,VOLUME,TURNOVER,TRADES,DELIVERABLE VOLUME,%DELIVERBLE,year,month,day,dayofweek_num
0,2016-01-01,ASIANPAINT,EQ,883.55,882.0,885.60,876.9,879.7,878.75,880.60,294006,2.589019e+13,7988,149531,0.5086,2016,1,1,4
1,2016-01-04,ASIANPAINT,EQ,878.75,876.2,892.45,871.7,880.5,880.80,881.96,709103,6.254040e+13,28981,337383,0.4758,2016,1,4,0
2,2016-01-05,ASIANPAINT,EQ,880.80,881.3,905.00,881.3,901.4,900.65,898.75,1372748,1.233761e+14,52028,533231,0.3884,2016,1,5,1
3,2016-01-06,ASIANPAINT,EQ,900.65,904.5,911.90,884.0,885.3,886.40,897.11,1716479,1.539873e+14,51374,1008881,0.5878,2016,1,6,2
4,2016-01-07,ASIANPAINT,EQ,886.40,883.0,891.55,869.0,871.1,872.35,878.39,990333,8.698976e+13,26249,477288,0.4819,2016,1,7,3


In [23]:
df_inp.SYMBOL.nunique()

30

In [24]:
df_inp.DATE.agg(['min','max'])

min   2016-01-01
max   2021-06-15
Name: DATE, dtype: datetime64[ns]

In [10]:
## day based features - ordinal data
df_inp['year']=df_inp['DATE'].dt.year 
df_inp['month']=df_inp['DATE'].dt.month 
df_inp['day']=df_inp['DATE'].dt.day

df_inp['dayofweek_num']=df_inp['DATE'].dt.dayofweek  

In [ ]:
## entity embedding for categoricals

## 

In [ ]:
## data processing
start_date = pd.to_datetime('2016-10-01')
end_date = pd.to_datetime('2018-10-01')
forecast_steps = 5
valid_period = 120 # num days
 

# level of forecast
level_foreca = 'SYMBOL'
list_symbols = df_inp[level_foreca].unique().tolist()

static features = ['SYMBOL','']
dynamic_features = 